In [24]:
import torch
import pandas as pd
df = pd.read_csv("data/raw/train.csv", index_col=0)

In [41]:
preds = torch.tensor(df.sample(128)['difficulty_average'].tolist()).unsqueeze(-1)
# difficulty = preds
difficulty = torch.tensor(df.sample(128)['difficulty_average'].tolist()).unsqueeze(-1)

In [42]:
def top1_top3_accuracy(preds, difficulty):
    diff = (preds - difficulty).abs()
    num_samples = diff.size(0)
    top1_guesses = (diff < 1).long()
    top3_guesses = (diff < 3).long()
    return top1_guesses.sum()/num_samples, top3_guesses.sum() / num_samples
    

In [43]:
top1_top3_accuracy(preds, difficulty)

(tensor(0.1328), tensor(0.3125))

In [40]:
for i in range(preds.size(0)):
    print(int(preds[i].item()), int(difficulty[i].item()))

22 14
16 18
25 15
19 25
13 21
17 22
20 22
13 20
25 22
18 18
12 20
18 13
20 24
17 15
20 26
22 20
20 15
17 23
21 22
23 21
24 20
17 20
19 21
24 28
17 28
11 20
22 26
21 23
22 19
16 24
21 25
19 20
23 27
12 16
22 15
19 19
24 15
15 15
17 17
14 10
18 24
19 19
15 14
22 16
23 13
23 25
13 26
11 22
17 21
11 17
14 21
24 16
19 19
23 26
16 16
18 18
13 18
24 19
21 16
14 15
19 19
15 17
16 20
21 20
23 20
20 20
25 16
21 21
14 13
16 10
18 23
15 16
29 21
24 18
11 10
13 21
24 27
23 11
21 18
19 22
22 20
23 18
20 11
19 22
13 17
27 21
21 19
10 25
16 12
16 21
16 13
16 22
19 21
26 23
27 22
28 19
19 22
13 22
17 21
17 20
18 12
11 20
23 12
11 13
20 14
16 11
21 21
20 22
11 11
13 21
13 21
24 13
16 20
14 24
15 18
22 20
23 24
24 25
16 12
20 10
22 22
24 24
22 13
13 13
21 16
10 18
10 18
10 21
